<img src="advection-diffusion-1D.png" align="center"/>

# Lab 2A: Advection-Diffusion equation (1D)

In this lab, we will introduce the Dedalus solver to explores some concepts from PDEs: 

- Constructing basis functions and domains
- Boundary conditions and initial value problems
- Formulating the problem and writing equations in Dedalus

The example we will be looking at is the advection diffusion equation. 

To use this notebook, you will need Python 3 and the latest version of [Dedalus](http://dedalus-project.org/) installed on your local machine. 

# Background

## The Advection-Diffusion equation

In this lab, we will solve the advection-diffusion equation in 1 and 2 dimensions:

$$
\partial_t \, C + \mathbf{U} \cdot \nabla C = K \nabla^2 C,
$$

where $C$ is the concentration of some quantity, for example dye, $\mathbf{U}$ is the flow field, and $K$ is the diffusivity, which controls how fast the dye disappears. 

We will start off by looking at the 1D advection-diffusion equation, which we can write explicitly as

$$
\partial_t \, C + U \, \partial_x \, C = K \, \partial_x^2 \, C,
$$

We will study this equation in a domain $-L \le x \le L$ as it evolves for time $t \ge 0$. 

The field $C = C(x,t)$ is a function of space and time. In principle, both $U$ and $K$ could also be functions of space and/or time. For now we will just assume that they are constants. 

## Boundary conditions

The highest spatial derivative in this problem is $\partial_x^2 C$. As you will remember from your calculus courses, if you want to solve an equation with two spatial derivatives, you need to specify two boundary conditions. We have a number of choices of boundary condition we can use, for example

- Set the value of $C$ to be zero at the left and right boundaries 

$$
C (x=-L) = 0 \quad \mbox{and} \quad C (x=L) = 0.
$$

This is called the Dirichlet boundary condition.

- Set the _derivative_ of $C$ to be zero at the left and right boundaries

$$
\partial_x \, C (x=-L) = 0 \quad \mbox{and} \quad \partial_x \, C (x=L) = 0.
$$

This is called the zero flux or Neumann boundary condition.

- Set _periodic boundary conditions_, which is equivalent to 

$$
C(x=-L) = C(x=L) \quad \mbox{and} \quad \partial_x \, C (x=-L) = \partial_x \, C (x=L).
$$

The evolution of the concentration field will be strongly controlled by the choice of boundary conditions that we make. 

## Initital value problem

In this problem we will specify the initial concentration $C = C_0(x)$ at $t = 0$. In general, this will be a function of space. Here, we will choose a particular form for the initial concentration so that it is confined to the centre of the domain:

$$
C_0 (x) = \frac{1}{2 n} \log \left( 1 + \frac{\cosh^2 n}{\cosh^2 nx} \right).
$$

Varying $n$ makes the initial concentration narrow or smaller, as needed. In the next cell, we will plot different initial conditions to get a sense of how they depend on $n$.

**Don't forget to type SHIFT+ENTER (or click RUN from the menu above) to execute each cell.**

In [ ]:
# import libraries and plot figures in Jupyter window
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# plot initial concentration values C_0
n = 128
x = np.linspace(-np.pi,np.pi,n)[1:-1, None].flatten()

C1 = np.log( 1 + np.cosh(1.)**2 / np.cosh(1.*x)**2 )/(2*1.)
C2 = np.log( 1 + np.cosh(2.)**2 / np.cosh(2.*x)**2 )/(2*2.)
C5 = np.log( 1 + np.cosh(5.)**2 / np.cosh(5.*x)**2 )/(2*5.)
C10 = np.log( 1 + np.cosh(10.)**2 / np.cosh(10.*x)**2 )/(2*10.)
C20 = np.log( 1 + np.cosh(20.)**2 / np.cosh(20.*x)**2 )/(2*20.)

plt.plot(x,C1)
plt.plot(x,C2)
plt.plot(x,C5)
plt.plot(x,C10)
plt.plot(x,C20)
plt.ylim(0,1)

## Formulating the problem

We are now ready to formulate the problem in a form suitable for Dedalus. To do this, we will rewrite the advection diffusion equation in such a way that the highest derivative is first order. Specifically, instead of solving a _second order_ PDE for one field $C (x,t)$, we will solve two _first order_ PDEs for field $C (x,t)$ and its derivative $C_x = \partial_x \, C(x,t)$. 

To see this, let's write the advection diffusion equation in terms of $C$ and $C_x$: 

$$
\partial_t \, C + U \, C_x = K \, \partial_x \, C_x.
$$

In this equation, we have rewritten the advection term 

$$
U \, \partial_x C \rightarrow U \, C_x
$$ 

and the diffusion term 

$$
K \, \partial_x^2 C \rightarrow K \, \partial_x \, C_x.
$$

Clearly, this doesn't change the content of the PDE at all. However, we now need to include an additional PDE for the new field $C_x$. This just comes from the definition of $C_x$

$$
C_x = \partial_x\, C.
$$

Finally, we will need to specify boundary conditions. We can impose these boundary conditions on either $C$ (Dirichlet BC) or $C_x$ (Nuemann BC). We will explore this in the experiment.

# Experimental setup

## Dedalus

We are now ready to introduce Dedalus, which we will use to solve the advection diffusion equation. 

We import the `dedalus` library in the same way that we did for `numpy` and `matplotlib`: via the `import ... as ...` command. In this case, we'll use `de` as the shorthand for `dedalus`. 

We will also import a couple of other useful libraries: some extra tools (`flow_tools`) for Dedalus, and a library called `time` that will tell us how much time has elapsed since we started the calculation. 

Finally, we'll call some commands so we can suppress some logging messages. 

In [ ]:
from dedalus import public as de
from dedalus.extras import flow_tools
import time
import logging
logger = logging.getLogger(__name__)

## Bases and Domains

In Dedalus, "Domain objects" represent physical domains which are created by combining bases ($x$, $y$, etc). We create bases in Dedalus by specifying a set of orthogonal basis functions, for example, sines and cosines (Fourier) or polynominals (Chebyshev). 

The choice of basis functions depends on the symmetry of the problem: for a problem that is periodic in the $x$-direction, for example, we choose sines and cosines (Fourier) for the $x$ basis functions. If the problem is _not_ periodic in the $y$ direction, polynomials (Chebyshev) are a better choice for the $y$ basis functions. 

To build a domain, we tell Dedalus the name of the basis, the number of gridpoints, and the interval. We then give Dedalus a list of the bases and specify the datatype. 

In [ ]:
# Parameters
L = np.pi # 2*L = length of domain
nx = 64   # number of gridpoints
K = 1.    # diffusivity
U0 = 0.   # velocity magnitude

# Create bases and domain
x_basis = de.Chebyshev('x', nx, interval=(-L,L))
domain = de.Domain([x_basis], grid_dtype=np.float64)

## Initial value problem

We need to specify what _kind_ of equation we are solving. This problem is an _initial value problem_, which we call using the command `IVP` from the Dedalus library. We also need to tell Dedalus the domain (i.e. the $x$ basis), the dynamical variables ($C$, $Cx$), and any parameters ($K$, $U0$) or functions ($U(x)$) that appear in the problem. 

In [ ]:
# Formulate the initial value problem
problem = de.IVP(domain, variables=['C','Cx'])

# Set parameters (diffusivity and flow speed)
problem.parameters['K'] = K
problem.parameters['U0'] = U0
problem.substitutions['U'] = 'U0'

## Formulating the problem

Systems of differential equations in Dedalus are represented in the form:

$$\mathcal{M} \cdot \partial_t \mathcal{X} + \mathcal{L} \cdot \mathcal{X} = \mathcal{F}$$

where $\mathcal{X}$ is a state-vector of fields, $\mathcal{M}$ and $\mathcal{L}$ are matrices of linear differential operators, and $\mathcal{F}$ is a set of generally nonlinear expressions.  This generalized form accomodates prognostic equations, diagnostic constraints, and boundary conditions.

Dedalus includes a symbolic parser that takes equations and boundary conditions specified in plain text, and manipulates them into the above matrix form.  This form requires the equations to be first-order in time and space derivatives, and must only contain linear terms on the left-hand-side. 

Since we are dealing with a set of linear first order differential equations, we have already written the problem in the right form. However, we will also consider cases when the velocity can vary in space, $U(x)$. So we will treat the advection term $U(x) \, C_x$ as a nonlinear term and move it to the right-hand side:

$$
\partial_t C - K \, \partial_x \, C_x = - U \, C_x,
$$

$$
C_x - \partial_x \, C = 0.
$$

## Equation entry

Equations and boundary conditions are then entered in plain text expressions that can be made up of
* The variables, parameters, and substitutions defined in the problem
* The axis names (`'x'` here), representing the individual basis grids
* Time and temporal derivatives as `'t'` and `'dt'`
* The differential operators for each basis, named as e.g. `'dx'`
* Differentiation, integration, and interpolation are entered using  `'d'`, `'integ'`, and `'interp'`
* Simple mathematical functions (logarithmic and trigonometric), e.g. `'sin'`, `'exp'`, ...

In addition we will specify the values of the the problem variables at the boundaries. For periodic boundary conditions, we don't need to do this explicitly: our choice of sines and cosines as basis functions already takes care of this for us. 

For more complex boundary conditions, we can use `'left'` and  to indicate the end points. Here left and right refer to position on the interval. So for the $x$-axis, `'left'` refers to $x = -L$ and `'right'` refers to $x=L$. 

Let's see how to enter the equations and boundary conditions for our problem.

In [ ]:
problem.add_equation("dt(C) - K*dx(Cx) = - U*Cx")
problem.add_equation("Cx - dx(C) = 0")

problem.add_bc("left(C) = 0")
problem.add_bc("right(C) = 0")

## Building a solver

Solvers are simply build using the `problem.build_solver` method. For IVPs, we select a timestepping method when building the solver. In this problem, we will use an RK222 timestepping method, but others are available. 

For IVPs, stop criteria for halting time evolution are specified by setting the `stop_iteration`, `stop_wall_time` (seconds since simulation started running), and/or `stop_sim_time` attributes on the solver.  

Let's stop after the model time reaches 1, or the solver takes 30 minutes. 

In [ ]:
# Build solver
solver = problem.build_solver(de.timesteppers.RK222)
logger.info('Solver built')

# timesteps
T  = 1
dt = 1/2000

# Integration parameters
solver.stop_sim_time = T
solver.stop_wall_time = 30 * 60.
solver.stop_iteration = np.inf

## Setting initial conditions

For IVPs, initial conditions are set by directly modifying the state variable data before running a simulation.

The initial condition itself will be applied to the concentration field, $C$. 

In [ ]:
# Get the bases from the object "domain" and the state variables from the object "solver"
x = domain.grid(0)
C = solver.state['C']
Cx = solver.state['Cx']

n = 20
C['g'] = np.log(1 + np.cosh(n)**2/np.cosh(n*x)**2) / (2*n)
C.differentiate(0, out=Cx)

C.set_scales(1, keep_data=True)
plt.plot(x,C['g'])

In [ ]:
import time

# Store data for final plot
C.set_scales(1, keep_data=True)
C_list = [np.copy(C['g'])]
t_list = [solver.sim_time]

## Solving the problem

The logic controlling the main-loop of a Dedalus simulation occurs explicitly in the simulation script.  The `solver.ok` property can be used to halt an evolution loop once any of the specified stopping criteria have been met.  Let's timestep our problem until a halting condition is reached, copying the grid values of `C` every few iterations.  This should take less than a minute on most machines.

In [ ]:
# Main loop
while solver.ok:
    solver.step(dt)
    if solver.iteration % 20 == 0:
        C.set_scales(1, keep_data=True)
        C_list.append(np.copy(C['g']))
        t_list.append(solver.sim_time)
    if solver.iteration % 100 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
# Make plot of C
plt.plot(x,C_list[0])
plt.plot(x,C_list[20])
plt.plot(x,C_list[40])
plt.plot(x,C_list[60])
plt.plot(x,C_list[80])
plt.plot(x,C_list[100])
plt.ylim(0,1)

In [ ]:
# plot average of C as a function of time
Cavg_list = np.sum(C_list[0])/nx

for i in range(len(C_list)):
    Cavg_list = np.append(Cavg_list,np.sum(C_list[i])/nx)
    
# Make plot of C average
t_plot = np.linspace(0,T,len(Cavg_list))
plt.plot(t_plot,Cavg_list)
plt.ylim(0,1)

In [ ]:
# plot variance of C as a function of time
Cvar_list = np.sum(C_list[0]**2)

for i in range(len(C_list)):
    Cvar_list = np.append(Cvar_list,np.sum(C_list[i]**2))
    
# Make plot of C variance
t_plot = np.linspace(0,T,len(Cvar_list))

plt.plot(t_plot,Cvar_list)

In [ ]:
import matplotlib.pyplot as plt

from dedalus.extras.plot_tools import quad_mesh, pad_limits

t_plot = np.linspace(0,T,len(t_list))

xmesh, ymesh = quad_mesh(x=x, y=t_plot)

fig, ax = plt.subplots(figsize=(12,12))

ax.pcolormesh(xmesh, ymesh, C_list, cmap='RdBu_r',clim=(-0.1,0.1))
ax.axis(pad_limits(xmesh, ymesh))
ax.set_xlabel('space (x)')
ax.set_ylabel('time (t)')
ax.set_title('$surface height$')
fig.savefig('space_time_diagram.png')

## Now try it yourself

Use this notebook to experiment with the advection diffusion problem. Add your observations, comments, and calculations to the notebook by double clicking the cells to enter new text. You can then press **shift+enter** to turn the cell back into readable form. 

When you make changes to the problem parameters or initial conditions, you will need to rebuild the solver. To do this, just re-run each cell from the beginning. 

- Modify the value the diffusivity $K$ to see how it changes the resulting concentration field and the variance. Does increasing $K$ make the concentration decay faster or slower? Why?

- Change the boundary conditions so that the _deriviatives_ of $C$ (rather than $C$ itself are zero at the left and right boundaries. That is, change the boundary conditions from `left(C) = 0` and `right(C) = 0` to `left(Cx) = 0` and `right(Cx) = 0`. (These are called zero-flux boundary conditions.)

- How do the new boundary conditions change the evolution of the concentration field? What about the spatial average? 

- Add a velocity field by making the velocity magnitude $U_0 = 1.$. How does this change the subsequent evolution of the concentration field? 

- You can add more complex velocity fields by modifying the substitution used for `U`. Try changing it so that `U = -U0*x`. How does the evolution of $C$ change? 

- Now try changing the velocity field to `U = U_0*x`. What does this do? 